In [1]:
import arxiv

In [2]:
client = arxiv.Client()

search = arxiv.Search(
    query = "quantum",
    max_results= 10,
    sort_by = arxiv.SortCriterion.SubmittedDate
)

In [18]:
results = list(client.results(search))

In [20]:
for r in results:
    print(r.title, ', '.join([author.name for author in r.authors]))

Non-equilibrium dynamics of symmetry-resolved entanglement and entanglement asymmetry: Exact asymptotics in Rule 54 Katja Klobas
Non-equilibrium dynamics of charged dual-unitary circuits Alessandro Foligno, Pasquale Calabrese, Bruno Bertini
A concrete construction of a topological operator in factorization algebras Masashi Kawahira
Lanczos for lattice QCD matrix elements Daniel C. Hackett, Michael L. Wagman
Properties of Krylov state complexity in qubit dynamics Siddharth Seetharaman, Chetanya Singh, Rejish Nath
Shadow Hamiltonian Simulation Rolando D. Somma, Robbie King, Robin Kothari, Thomas O'Brien, Ryan Babbush
Spurious Solar-Wind Effects on Acceleration Noise in LISA Pathfinder Arnold Yang, Indie Desiderio-Sloane, Grant David Meadors
Engineering a multi-level bath for transmon with three-wave mixing and parametric drives Xi Cao, Maria Mucci, Gangqiang Liu, David Pekker, Michael Hatridge
Energy Transport Among Highly-Polarized Atoms Catherine D. Opsahl, Yuan Jiang, Samantha A. Grub

In [22]:
type(r.links)

datetime.datetime

In [6]:
for r in client.results(search):
    print(r.title)

Non-equilibrium dynamics of symmetry-resolved entanglement and entanglement asymmetry: Exact asymptotics in Rule 54
Non-equilibrium dynamics of charged dual-unitary circuits
A concrete construction of a topological operator in factorization algebras
Lanczos for lattice QCD matrix elements
Properties of Krylov state complexity in qubit dynamics
Shadow Hamiltonian Simulation
Spurious Solar-Wind Effects on Acceleration Noise in LISA Pathfinder
Engineering a multi-level bath for transmon with three-wave mixing and parametric drives
Energy Transport Among Highly-Polarized Atoms
Minimal Quantum Circuits for Simulating Fibonacci Anyons
